In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import os

import seaborn as sns

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
filepath_isotypes = '/networks/cavd/VDCs/Schief/Schief_856-G002/SkinReactions/data/Glycan_array_Scripps/processed_data/DRAFT_CAVD_G002_Glycan_Microarray_data_processed_2024-10-16.txt'
df_glycan_isotypes = pd.read_csv(filepath_isotypes, sep="\t")

/tmp/ipykernel_44970/4284450680.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_glycan_isotypes = pd.read_csv(filepath_isotypes, sep="\t")


In [4]:
usecols = ['sample_id',
           'isotype', 
           'ptid', 
           'study_week',
           'spot_name', 
           'glycan_m_number', 
           'background_subtraced_mean_signal']

df = df_glycan_isotypes[usecols]

In [5]:
df.sample_id = df.sample_id.astype(str)

In [6]:
def centered_mean(x):
    if len(x) >= 6:
        return np.mean(np.sort(x)[1:-1])
    else:
        return np.mean(x)

df['centered_mean'] = df.groupby(['isotype','sample_id','glycan_m_number'])[['background_subtraced_mean_signal']].transform(centered_mean)
        
# df['normal_mean'] = df.groupby(['isotype','sample_id','glycan_m_number'])[['background_subtraced_mean_signal']].transform("mean")
# df['count_of_applicable'] = df.groupby(['isotype','sample_id','glycan_m_number'])[['background_subtraced_mean_signal']].transform(lambda x: len(x))
# df['centered_mean'] = df.groupby(['isotype','sample_id','glycan_m_number'])[['background_subtraced_mean_signal']].transform(lambda x: np.mean(np.sort(x)[1:-1]))
# df = df.drop(columns='background_subtraced_mean_signal').drop_duplicates()

df = df.drop(columns='background_subtraced_mean_signal').drop_duplicates()

df = df.reset_index(drop=True)

calc_responses = df.copy()

calc_responses['threshold'] = 100
calc_responses['response_flag'] = calc_responses.centered_mean > 100
calc_responses["count_of_responses"] = calc_responses.groupby(['isotype','study_week','glycan_m_number'])[['response_flag']].transform('sum')
calc_responses["prop_of_responses"] = calc_responses.groupby(['isotype','study_week','glycan_m_number'])[['response_flag']].transform(lambda x: x.sum()/len(x))

calc_responses["overall_response_rate"] = calc_responses.groupby(['isotype','glycan_m_number'])[['response_flag']].transform(lambda x: x.sum()/len(x))
calc_responses = calc_responses.sort_values(by='overall_response_rate', ascending=False)

In [81]:
savedir = '/networks/vtn/lab/SDMC_labscience/operations/documents/templates/assay/template_testing/plots_11_07_2024/'

isotype="IgG"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()

t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 4
    M = 5
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    # fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(9,N*2.5), sharex=True, sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                if max([len(i) for i in data]) > 2:
                    print(f"{20*(t-1) + k} - {glycan}")
    remaining_glycans = remaining_glycans[20:]


0 - 589 Sp24
1 - 387 Sp20
2 - Yeast Mannan Sigma M-3640
3 - 104 Sp8
4 - Galactomannan DAVANT (160102) Pro-Pharmacenti
5 - 105 Sp0
6 - 395 Sp20
7 - 397 Sp20
8 - 508 Sp21
9 - 394 Sp20
10 - 517 Sp24
11 - 495Sp21
12 - 587 Sp24
13 - 600 Sp24
14 - 588 Sp24


In [77]:
g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]

In [78]:
[len(i) for i in data]

[0, 0, 1]

In [ ]:
# >= 3
# igg - 15
# ige - 9
# igm - 2

# permissive
# igg - 28
# ige - 27
# igm - 11

In [11]:
savedir = '/networks/vtn/lab/SDMC_labscience/operations/documents/templates/assay/template_testing/plots_11_07_2024/'

isotypes = ['IgE','IgM','IgG']
useticks = {}
useticks["IgG"] = [100, 1_000, 10_000, 15_000]
useticks["IgE"] = [100, 1_000, 1_700]
useticks["IgM"] = [100, 1_000]

ylim = {}
ylim['IgG'] = (100, 15_000)
ylim['IgE'] = (100, 1_700)
ylim['IgM'] = (100, 1_000)

In [33]:
g = calc_responses.loc[(calc_responses.glycan_m_number=='589 Sp24') & (calc_responses.isotype=='IgG') & (calc_responses.centered_mean >= 100)]
g.loc[g.study_week=="Wk 8"].shape

(31, 12)

In [34]:
data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]

In [123]:
9, 3*2.5

(9, 7.5)

In [1]:
isotype = "IgG"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()
glycan_ordering = glycan_ordering[:28]

t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 6
    M = 5
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(9,N*2.5), sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
            
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                Ns = [str(len(i)) for i in data]
                ax[i,j].boxplot(data)
                ax[i,j].set_xticks(ticks=np.arange(1,4), labels=[f"N={Ns[i]}\n{weeks[i]}" for i in range(3)])
                ax[i,j].set_yscale("log")
                ax[i,j].set_ylim(ylim[isotype])
                ax[i,j].set_yticks(useticks[isotype])

                if len(glycan) > 20:
                    usethis = str(glycan.replace(' ','\n'))
                    ax[i,j].set_title(usethis, fontsize=10)
                else:
                    ax[i,j].set_title(f"{glycan}", fontsize=10)
                if j==0:
                    ax[i,j].set_ylabel("centered mean of\nbackground-normalized signal")
    
    fig.tight_layout()
    plt.suptitle(f"{isotype} - glycans w/ any responders", fontsize=14, y=1.005)
    plt.savefig(savedir + f"{isotype}_boxplots{t}.png", dpi=320, format='png', transparent=False, bbox_inches='tight', pad_inches=0.3)
    remaining_glycans = remaining_glycans[28:]

# imgs = [f"{isotype}_boxplots{i}.png" for i in np.arange(1,2)]

# images = [
#     Image.open(savedir + f)
#     for f in imgs
# ]

# pdf_path = savedir + f"{isotype}_glycan_boxplots_any.pdf"
    
# images[0].save(
#     pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
# )

In [2]:
isotype = "IgG"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()
glycan_ordering = glycan_ordering[:15]

t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 3
    M = 5
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(9,N*2.5), sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
            
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                Ns = [str(len(i)) for i in data]
                ax[i,j].boxplot(data)
                ax[i,j].set_xticks(ticks=np.arange(1,4), labels=[f"N={Ns[i]}\n{weeks[i]}" for i in range(3)])
                ax[i,j].set_yscale("log")
                ax[i,j].set_ylim(ylim[isotype])
                ax[i,j].set_yticks(useticks[isotype])

                if len(glycan) > 20:
                    usethis = str(glycan.replace(' ','\n'))
                    ax[i,j].set_title(usethis, fontsize=10)
                else:
                    ax[i,j].set_title(f"{glycan}", fontsize=10)
                if j==0:
                    ax[i,j].set_ylabel("centered mean of\nbackground-normalized signal")
    
    fig.tight_layout()
    plt.suptitle(f"{isotype} - glycans w/ >=3 responders", fontsize=14, y=1.005)
    plt.savefig(savedir + f"{isotype}_boxplots{t}_min3.png", dpi=320, format='png', transparent=False, bbox_inches='tight', pad_inches=0.3)
    remaining_glycans = remaining_glycans[15:]

# plt.show()
# imgs = [f"{isotype}_boxplots{i}_min3.png" for i in np.arange(1,2)]

# images = [
#     Image.open(savedir + f)
#     for f in imgs
# ]

# pdf_path = savedir + f"{isotype}_glycan_boxplots_min3.pdf"
    
# images[0].save(
#     pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
# )

In [3]:
isotype = "IgE"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()
glycan_ordering = glycan_ordering[:27]

t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 6
    M = 5
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(9,N*2.7), sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
            
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                Ns = [str(len(i)) for i in data]
                ax[i,j].boxplot(data)
                ax[i,j].set_xticks(ticks=np.arange(1,4), labels=[f"N={Ns[i]}\n{weeks[i]}" for i in range(3)])
                ax[i,j].set_yscale("log")
                ax[i,j].set_ylim(ylim[isotype])
                ax[i,j].set_yticks(useticks[isotype])

                if len(glycan) > 20:
                    usethis = str(glycan.replace(' ','\n'))
                    ax[i,j].set_title(usethis, fontsize=10)
                else:
                    ax[i,j].set_title(f"{glycan}", fontsize=10)
                if j==0:
                    ax[i,j].set_ylabel("centered mean of\nbackground-normalized signal")
    
    # ax[N-1,M-1].set_xticks(ticks=np.arange(1,4), labels=weeks)
    fig.tight_layout()
    plt.suptitle(f"{isotype} - glycans w/ any responders", fontsize=14, y=1.005)
    plt.savefig(savedir + f"{isotype}_boxplots{t}.png", dpi=320, format='png', transparent=False, bbox_inches='tight', pad_inches=0.3)
    remaining_glycans = remaining_glycans[28:]


In [4]:
isotype = "IgE"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()
glycan_ordering = glycan_ordering[:9]

t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 2
    M = 5
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(9,N*2.7), sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
            
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                Ns = [str(len(i)) for i in data]
                ax[i,j].boxplot(data)
                ax[i,j].set_xticks(ticks=np.arange(1,4), labels=[f"N={Ns[i]}\n{weeks[i]}" for i in range(3)])
                ax[i,j].set_yscale("log")
                ax[i,j].set_ylim(ylim[isotype])
                ax[i,j].set_yticks(useticks[isotype])

                if len(glycan) > 20:
                    usethis = str(glycan.replace(' ','\n'))
                    ax[i,j].set_title(usethis, fontsize=10)
                else:
                    ax[i,j].set_title(f"{glycan}", fontsize=10)
                if j==0:
                    ax[i,j].set_ylabel("centered mean of\nbackground-normalized signal")
    
    # ax[N-1,M-1].set_xticks(ticks=np.arange(1,4), labels=weeks)
    fig.tight_layout()
    plt.suptitle(f"{isotype} - glycans w/ >=3 responders", fontsize=14, y=1.02)
    plt.savefig(savedir + f"{isotype}_boxplots{t}_min3.png", dpi=320, format='png', transparent=False, bbox_inches='tight', pad_inches=0.3)
    remaining_glycans = remaining_glycans[9:]


# imgs = [f"{isotype}_boxplots{i}_min3.png" for i in np.arange(1,2)]

# images = [
#     Image.open(savedir + f)
#     for f in imgs
# ]

# pdf_path = savedir + f"{isotype}_glycan_boxplots_min3.pdf"
    
# images[0].save(
#     pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
# )

In [5]:
isotype = "IgM"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()
glycan_ordering = glycan_ordering[:11]

t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 3
    M = 5
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(9,N*2.8), sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
            
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                Ns = [str(len(i)) for i in data]
                ax[i,j].boxplot(data)
                ax[i,j].set_xticks(ticks=np.arange(1,4), labels=[f"N={Ns[i]}\n{weeks[i]}" for i in range(3)])
                ax[i,j].set_yscale("log")
                ax[i,j].set_ylim(ylim[isotype])
                ax[i,j].set_yticks(useticks[isotype])

                if len(glycan) > 20:
                    usethis = str(glycan.replace(' ','\n'))
                    ax[i,j].set_title(usethis, fontsize=10)
                else:
                    ax[i,j].set_title(f"{glycan}", fontsize=10)
                if j==0:
                    ax[i,j].set_ylabel("centered mean of\nbackground-normalized signal")
    
    # ax[N-1,M-1].set_xticks(ticks=np.arange(1,4), labels=weeks)
    fig.tight_layout()
    plt.suptitle("IgM - glycans w/ any responders", fontsize=14, y=1.05)
    plt.savefig(savedir + f"{isotype}_boxplots{t}_any.png", dpi=320, format='png', transparent=False, bbox_inches='tight', pad_inches=0.3)
    remaining_glycans = remaining_glycans[12:]


In [8]:
isotype = "IgM"
glycan_ordering = calc_responses.loc[calc_responses.isotype==isotype,['glycan_m_number','overall_response_rate']].drop_duplicates().glycan_m_number.tolist()
glycan_ordering = glycan_ordering[:2]


t = 0
remaining_glycans = glycan_ordering
while len(remaining_glycans)>0:
    t += 1
    N = 1
    M = 2
    weeks = ["Wk 0", "Wk 8", "Wk 10"]
    
    fig, ax = plt.subplots(nrows=N, ncols=M, figsize=(6,N*3.5), sharey=True)
    for i in range(N):
        for j in range(M):
            k = i*M + j
            if k < len(remaining_glycans):
                glycan = remaining_glycans[k]
            
                g = calc_responses.loc[(calc_responses.glycan_m_number==glycan) & (calc_responses.isotype==isotype) & (calc_responses.centered_mean >= 100)]
                data = [list(g.loc[(g.study_week==w)].centered_mean) for w in weeks]
                Ns = [str(len(i)) for i in data]
                ax[j].boxplot(data)
                ax[j].set_xticks(ticks=np.arange(1,4), labels=[f"N={Ns[i]}\n{weeks[i]}" for i in range(3)])
                ax[j].set_yscale("log")
                ax[j].set_ylim(ylim[isotype])
                ax[j].set_yticks(useticks[isotype])

                if len(glycan) > 20:
                    usethis = str(glycan.replace(' ','\n'))
                    ax[j].set_title(usethis, fontsize=10)
                else:
                    ax[j].set_title(f"{glycan}", fontsize=10)
                if j==0:
                    ax[j].set_ylabel("centered mean of\nbackground-normalized signal")
    
    # ax[N-1,M-1].set_xticks(ticks=np.arange(1,4), labels=weeks)
    fig.tight_layout()
    plt.suptitle(f"{isotype} - glycans w/ >=3 responders", fontsize=14, y=1.05)
    plt.savefig(savedir + f"{isotype}_boxplots{t}_min3.png", dpi=320, format='png', transparent=False, bbox_inches='tight', pad_inches=0.3)
    remaining_glycans = remaining_glycans[12:]

In [92]:
g = calc_responses.loc[(calc_responses.isotype=="IgM") & (calc_responses.centered_mean >= 100)]


In [137]:
imgs = [f"{isotype}_boxplots1.png" for isotype in ["IgG","IgE","IgM"]]

images = [
    Image.open(savedir + f)
    for f in imgs
]

pdf_path = savedir + f"glycan_boxplots_any.pdf"

images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

In [138]:
imgs = [f"{isotype}_boxplots1_min3.png" for isotype in ["IgG","IgE","IgM"]]

images = [
    Image.open(savedir + f)
    for f in imgs
]

pdf_path = savedir + f"glycan_boxplots_min3.pdf"

images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)